**PySpark no Google Colab**

In [16]:
# instalar as dependências
# baixando a versão mais recente do java8;
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#download via stp do hadoop
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
#extraindo o arquivo tgz
!tar xf /content/spark-3.3.1-bin-hadoop3.tgz
#instalando o  Findspark que é responsavel para que as variáveis ​​do ambiente sejam definidas corretamente e o pyspark seja importado.
!pip install -q findspark

In [15]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

# utilizando o findspark para que o pyspark seja "importável"
import findspark
findspark.init('spark-3.3.1-bin-hadoop3')

**SparkSession**

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()

df = spark.read.csv(path='/mnt/clientes.csv',sep=';',header='true',encoding='latin1',inferSchema='true')

df.show()


+----------+--------------------+-------+--------+--------------------+------------+------+-------+--------+----------------+---------+----------+
|cliente_id|          nm_cliente| nr_cpf|dt_nasci|         complemento| nr_telefone|   cep|cep_mae| salario|desconto_salario|cod_banco|nome_banco|
+----------+--------------------+-------+--------+--------------------+------------+------+-------+--------+----------------+---------+----------+
|      1820|   Vitor Hugo da Luz| 108908|30031986|             179,...|00-0910-2646|400487|   null|71993.34|              21|       33| Santander|
|      2275|         Ian Moreira| 163362|20031983|             97,c...|00-1365-3969|400487|   null|34812.34|             208|       33| Santander|
|      2730|        Nicole Silva| 217816|26041989|             194,...|00-1820-5292|400487|   null|13884.47|             219|       33| Santander|
|      3185|       Enrico Araújo| 272270|18111982|             304,...|00-2275-6615|  null| 400487|88561.84|          

**.Select()**

Ele recebe de argumento 2 tipos de parametros.

1º: O nome da coluna em uma string

2º: Um objeto de coluna.

Para criar um objeto de coluna, utilizamos sempre os Métodos de F.

In [23]:
help(df.select)

Help on method select in module pyspark.sql.dataframe:

select(*cols: 'ColumnOrName') -> 'DataFrame' method of pyspark.sql.dataframe.DataFrame instance
    Projects a set of expressions and returns a new :class:`DataFrame`.
    
    .. versionadded:: 1.3.0
    
    Parameters
    ----------
    cols : str, :class:`Column`, or list
        column names (string) or expressions (:class:`Column`).
        If one of the column names is '*', that column is expanded to include all columns
        in the current :class:`DataFrame`.
    
    Examples
    --------
    >>> df.select('*').collect()
    [Row(age=2, name='Alice'), Row(age=5, name='Bob')]
    >>> df.select('name', 'age').collect()
    [Row(name='Alice', age=2), Row(name='Bob', age=5)]
    >>> df.select(df.name, (df.age + 10).alias('age')).collect()
    [Row(name='Alice', age=12), Row(name='Bob', age=15)]



**F.col('')**

O retorno é um objeto de coluna.

Este objeto aponta para alguma coluna do dataframe.


Atenção!!!
Priorizar o uso de um objeto de coluna em vez de passar uma string...

Pois é mais comum os métodos do dataframe aceitarem objetos de coluna do que 
uma string com o nome da coluna.


In [27]:
# df.select(
#     F.col('nm_cliente'),
#     F.col('salario')

# ).show()


df.select(
    'nm_cliente',
    'salario'
).show()

+--------------------+--------+
|          nm_cliente| salario|
+--------------------+--------+
|   Vitor Hugo da Luz|71993.34|
|         Ian Moreira|34812.34|
|        Nicole Silva|13884.47|
|       Enrico Araújo|88561.84|
|       Lara Nogueira|32141.87|
|Carlos Eduardo da...|52535.62|
|     Larissa Correia|22055.04|
|       Rafaela Viana|14216.09|
|       Raul Carvalho|11144.66|
|      Lorenzo Novaes|11546.18|
|      Alexia Ribeiro|31811.39|
|    Bárbara Nogueira| 1715.53|
|   Stephany da Rocha|87833.26|
|      Caroline Pires| 92904.9|
|        Kaique Jesus| 5012.54|
|        Alice da Luz|77676.76|
|       Eduarda Costa|87644.14|
|        Helena Souza|26649.08|
|      Lorena Azevedo|47519.21|
|Maria Vitória da ...|76523.79|
+--------------------+--------+
only showing top 20 rows



**Realizar Soma entre colunas**

In [29]:
df.select(
    'nm_cliente',
    F.col('salario') + F.col('desconto_salario')
).show()

+--------------------+----------------------------+
|          nm_cliente|(salario + desconto_salario)|
+--------------------+----------------------------+
|   Vitor Hugo da Luz|                    72014.34|
|         Ian Moreira|                    35020.34|
|        Nicole Silva|                    14103.47|
|       Enrico Araújo|                    88573.84|
|       Lara Nogueira|                    32361.87|
|Carlos Eduardo da...|                    52684.62|
|     Larissa Correia|                    22219.04|
|       Rafaela Viana|                    14430.09|
|       Raul Carvalho|                    11208.66|
|      Lorenzo Novaes|                    11738.18|
|      Alexia Ribeiro|                    31872.39|
|    Bárbara Nogueira|                     1839.53|
|   Stephany da Rocha|                    87936.26|
|      Caroline Pires|                     93129.9|
|        Kaique Jesus|                     5044.54|
|        Alice da Luz|                    77794.76|
|       Edua

**Realizar Divisão entre colunas**

In [33]:
df.select(
    'nm_cliente',
    (
        F.col('salario') / F.col('desconto_salario')
        
        ).alias('sobra'),
    F.col('nm_cliente').alias('nome_completo')
).show()

+--------------------+------------------+--------------------+
|          nm_cliente|             sobra|       nome_completo|
+--------------------+------------------+--------------------+
|   Vitor Hugo da Luz|3428.2542857142857|   Vitor Hugo da Luz|
|         Ian Moreira|167.36701923076922|         Ian Moreira|
|        Nicole Silva| 63.39940639269406|        Nicole Silva|
|       Enrico Araújo| 7380.153333333333|       Enrico Araújo|
|       Lara Nogueira| 146.0994090909091|       Lara Nogueira|
|Carlos Eduardo da...| 352.5880536912752|Carlos Eduardo da...|
|     Larissa Correia| 134.4819512195122|     Larissa Correia|
|       Rafaela Viana| 66.43032710280374|       Rafaela Viana|
|       Raul Carvalho|       174.1353125|       Raul Carvalho|
|      Lorenzo Novaes| 60.13635416666667|      Lorenzo Novaes|
|      Alexia Ribeiro| 521.4981967213115|      Alexia Ribeiro|
|    Bárbara Nogueira|13.834919354838709|    Bárbara Nogueira|
|   Stephany da Rocha| 852.7500970873785|   Stephany da

**Realizar multiplicação entre colunas**

In [35]:
df.select(
    'nm_cliente',
    (F.col('salario') * F.col('desconto_salario')).alias('mult')
).show()

+--------------------+------------------+
|          nm_cliente|              mult|
+--------------------+------------------+
|   Vitor Hugo da Luz|        1511860.14|
|         Ian Moreira| 7240966.719999999|
|        Nicole Silva|3040698.9299999997|
|       Enrico Araújo|        1062742.08|
|       Lara Nogueira| 7071211.399999999|
|Carlos Eduardo da...| 7827807.380000001|
|     Larissa Correia|        3617026.56|
|       Rafaela Viana|3042243.2600000002|
|       Raul Carvalho|         713258.24|
|      Lorenzo Novaes|        2216866.56|
|      Alexia Ribeiro|        1940494.79|
|    Bárbara Nogueira|         212725.72|
|   Stephany da Rocha|        9046825.78|
|      Caroline Pires|      2.09036025E7|
|        Kaique Jesus|         160401.28|
|        Alice da Luz|        9165857.68|
|       Eduarda Costa|     1.323426514E7|
|        Helena Souza|         879419.64|
|      Lorena Azevedo|         5702305.2|
|Maria Vitória da ...|1913094.7499999998|
+--------------------+------------